In [11]:
# imports
import pymysql as mdb       # native python library, no C integration issues
import json
import boto3


In [5]:
# add constants
aws_secret_name = "dig-bio-index"
aws_secret_region = "us-east-1"


In [6]:
# method to retrive the secrets given name and region
def get_secret(secret_name, region_name):
    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    # In this sample we only handle the specific exceptions for the 'GetSecretValue' API.
    # See https://docs.aws.amazon.com/secretsmanager/latest/apireference/API_GetSecretValue.html
    # We rethrow the exception by default.

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        if e.response['Error']['Code'] == 'DecryptionFailureException':
            # Secrets Manager can't decrypt the protected secret text using the provided KMS key.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'InternalServiceErrorException':
            # An error occurred on the server side.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'InvalidParameterException':
            # You provided an invalid value for a parameter.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'InvalidRequestException':
            # You provided a parameter value that is not valid for the current state of the resource.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'ResourceNotFoundException':
            # We can't find the resource that you asked for.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
    else:
        # Decrypts secret using the associated KMS CMK.
        # Depending on whether the secret is a string or binary, one of these fields will be populated.
        if 'SecretString' in get_secret_value_response:
            secret = get_secret_value_response['SecretString']
        else:
            decoded_binary_secret = base64.b64decode(get_secret_value_response['SecretBinary'])

    return json.loads(secret)


In [12]:
# get the secret
aws_secret = get_secret(aws_secret_name, aws_secret_region)

print("the secret database is {}".format(aws_secret['dbname']))


the secret database is bio


In [15]:
# method to take in secret and retirn tables
def show_tables(secret):
    db = mdb.connect(secret['host'], secret['username'], secret['password'], secret['dbname'])
    sql = "show tables"
    cursor = db.cursor()
    table_list = []

    # execute
    cursor.execute(sql)

    # fetch
    for row in cursor:
        table_list.append(row[0])

    # return
    return table_list




In [16]:
# get the table list
tables = show_tables(aws_secret)

print("the tables are: {}".format(tables))

the tables are: ['AnnotatedRegions', 'Associations', 'Burden', 'DatasetAssociations', 'Datasets', 'Gene', 'Genes', 'GlobalEnrichment', 'PhenotypeAssociations', 'Regions', 'TopAssociations', 'Variant', 'Variants', '__Indexes']
